### Edgar Moises Hernandez-Gonzalez
#### 26/07/20
#### Clasificacion de imagenes motoras en EEG con CNN-2D + LSTM
##### Caracteristicas = Escalogramas CWT (3D)
##### Entrada de 90x384
##### Busqueda de cuadricula

In [1]:
from google.colab import drive

In [3]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
import pywt
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import (TimeDistributed, Conv2D, MaxPool2D, Flatten, LSTM,
                          Dense, Dropout)
from keras.optimizers import Adam
from keras.wrappers.scikit_learn import KerasClassifier
from keras import backend as K

Using TensorFlow backend.


In [4]:
# para que este script funcione debe de decir 'channels_last', si dice 'channels_first' no sirve
K.image_data_format()

'channels_last'

In [5]:
c3_train = pd.read_csv("/content/drive/My Drive/BCI-II-III/Datos/MIEEG_train.csv", header=None).iloc[:, 0:768]
c4_train = pd.read_csv("/content/drive/My Drive/BCI-II-III/Datos/MIEEG_train.csv", header=None).iloc[:, 1536:2304]
c3_test = pd.read_csv("/content/drive/My Drive/BCI-II-III/Datos/MIEEG_test.csv", header=None).iloc[:, 0:768]
c4_test = pd.read_csv("/content/drive/My Drive/BCI-II-III/Datos/MIEEG_test.csv", header=None).iloc[:, 1536:2304]

In [6]:
x_train = pd.concat([c3_train, c4_train], axis=1)
x_test = pd.concat([c3_test, c4_test], axis=1)

In [7]:
# x_train = pd.read_csv("/content/drive/My Drive/BCI-II-III/Datos/MIEEG_train.csv", header=None)
# x_test = pd.read_csv("/content/drive/My Drive/BCI-II-III/Datos/MIEEG_test.csv", header=None)
y_train = pd.read_csv("/content/drive/My Drive/BCI-II-III/Datos/etiquetas_train.csv", header=None)
y_test = pd.read_csv("/content/drive/My Drive/BCI-II-III/Datos/etiquetas_test.csv", header=None)

In [8]:
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

(140, 1536)
(140, 1536)
(140, 1)
(140, 1)


In [9]:
x = pd.concat([x_train, x_test])
y = pd.concat([y_train, y_test])

In [10]:
print(x.shape)
print(y.shape)

(280, 1536)
(280, 1)


In [11]:
# en caso de no tener el y_test se puede utilizar el x_test
n_samples = len(y)

print("n_samples:", n_samples)

n_samples: 280


In [12]:
# esto se podria calcular asi n_clases = len(np.unique(y)), la y puede ser train o test
n_classes = len(np.unique(y))

print("n_classes:", n_classes)

n_classes: 2


In [15]:
#unir 2 canales y 3 segundos
def unir_escalogramas_vertical(data):
  fs = 128
  escalas = pywt.scale2frequency('cmor3-3', np.arange(8,30.5,0.5)) / (1/fs)
  datos = np.zeros((data.shape[0],90,384))
  temporal = np.zeros((90,384))
  for i in range(data.shape[0]): #n muestras
    for j in range(2): #2 canales
      sig = data.iloc[i, j*768:(j+1)*768]
      sig = sig[32:416]
      # escalograma CWT
      coef, freqs = pywt.cwt(sig, escalas, 'cmor3-3', sampling_period = (1 / fs))
      temporal[j*45:(j+1)*45, :] = abs(coef)
    datos[i] = temporal
    if i % 100 == 0: #esto solo es para ver como avanza
      print(i)
  return datos

In [16]:
# unir_escalogramas_vertical_resize(data):
inicio = time.time()

x = unir_escalogramas_vertical(x)

fin = time.time()
print("Tiempo:", fin - inicio)

0
100
200
Tiempo: 3.2489185333251953


In [17]:
print(x.shape)

(280, 90, 384)


In [18]:
print(np.max(x))
print(np.min(x))
print(np.mean(x))

0.341911306993214
4.934162308476574e-06
0.01788077799724545


In [19]:
# convertir de 3D a 5D
x = x.reshape((x.shape[0], 1, x.shape[1], x.shape[2], 1))

print(x.shape)

(280, 1, 90, 384, 1)


In [20]:
def CNN_LSTM(n_unidades, n_neuronas):
  model = Sequential()
  model.add(TimeDistributed(Conv2D(8, (3,3), activation='relu', padding='same'),
                            input_shape=x.shape[1:]))
  model.add(TimeDistributed(MaxPool2D((2,2))))
  model.add(TimeDistributed(Conv2D(8, (3,3), activation='relu', padding='same')))
  model.add(TimeDistributed(MaxPool2D((2,2))))
  model.add(TimeDistributed(Flatten()))
  model.add(LSTM(n_unidades, activation='tanh', dropout=0.5))
  model.add(Dense(n_neuronas, activation='relu'))
  model.add(Dense(n_classes, activation='softmax'))

  optimizer = Adam(lr=1e-4)
  model.compile(optimizer = optimizer,
                loss = 'sparse_categorical_crossentropy',
                metrics = ['accuracy'])
  return model

In [21]:
clf = KerasClassifier(build_fn = CNN_LSTM, epochs=100, batch_size=36, verbose=0)

In [22]:
n_unidades = [4, 8, 16, 32]
n_neuronas = [16, 32, 64, 128, 256]
hiperparametros = dict(n_unidades = n_unidades,
                       n_neuronas = n_neuronas)

In [23]:
# 20 redes = 1222 seg
4*5

20

In [24]:
grid = GridSearchCV(clf, hiperparametros, cv=5)

In [25]:
inicio = time.time()

grid.fit(x, y)

fin = time.time()
print("Tiempo:", fin - inicio)

Tiempo: 1222.0694732666016


In [26]:
print(grid.best_params_)
print(grid.best_score_)

{'n_neuronas': 256, 'n_unidades': 4}
0.8214285731315613


In [27]:
means = grid.cv_results_['mean_test_score']
stds = grid.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, grid.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r"
        % (mean, std * 2, params))

0.786 (+/-0.039) for {'n_neuronas': 16, 'n_unidades': 4}
0.800 (+/-0.118) for {'n_neuronas': 16, 'n_unidades': 8}
0.804 (+/-0.120) for {'n_neuronas': 16, 'n_unidades': 16}
0.782 (+/-0.138) for {'n_neuronas': 16, 'n_unidades': 32}
0.800 (+/-0.097) for {'n_neuronas': 32, 'n_unidades': 4}
0.814 (+/-0.080) for {'n_neuronas': 32, 'n_unidades': 8}
0.782 (+/-0.142) for {'n_neuronas': 32, 'n_unidades': 16}
0.779 (+/-0.171) for {'n_neuronas': 32, 'n_unidades': 32}
0.779 (+/-0.114) for {'n_neuronas': 64, 'n_unidades': 4}
0.786 (+/-0.145) for {'n_neuronas': 64, 'n_unidades': 8}
0.775 (+/-0.135) for {'n_neuronas': 64, 'n_unidades': 16}
0.793 (+/-0.114) for {'n_neuronas': 64, 'n_unidades': 32}
0.800 (+/-0.109) for {'n_neuronas': 128, 'n_unidades': 4}
0.804 (+/-0.150) for {'n_neuronas': 128, 'n_unidades': 8}
0.807 (+/-0.086) for {'n_neuronas': 128, 'n_unidades': 16}
0.782 (+/-0.120) for {'n_neuronas': 128, 'n_unidades': 32}
0.821 (+/-0.126) for {'n_neuronas': 256, 'n_unidades': 4}
0.771 (+/-0.109) f

In [28]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime → "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Mon Jul 27 00:16:00 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.51.05    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   76C    P0    46W /  70W |   1661MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------